In [ ]:
# 必要なライブラリのインストール
# !pip install opencv-python scikit-image

In [1]:
import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import glob
import zipfile

In [ ]:
"""
#--------------------------------------------------
# 【提出に関して】kaggleのAPIキーを入れるのは無理なので、
# 一時的に実行可能な形にリファクタリングします。
# -------------------------------------------------

# Kaggleデータセットのダウンロード（初回のみ）
def download_kaggle_dataset():
       print("Kaggleデータセットをダウンロード中...")

    # Kaggle APIの設定
    !pip install kaggle
    !mkdir -p ~/.kaggle
    !cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

    # データセットをダウンロード
    !kaggle datasets download -d koryakinp/fingers

    # ZIPファイルを解凍
    !unzip -q fingers.zip -d /content/kaggle_fingers/

    print("ダウンロード完了")
"""

In [3]:
def load_image_resized(image_path, target_size=(64, 64)):
    """画像を読み込んでリサイズ"""
    image = cv2.imread(image_path)
    if image is None:
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, target_size)
    return resized.flatten()

In [ ]:
def load_your_dataset(base_folder):
    """データセットを読み込み"""
    print("データセットを読み込み中...")
    X = []
    y = []

    if not os.path.exists(base_folder):
        print(f"警告: {base_folder} が見つかりません")
        return np.array([]), np.array([])

    subfolders = [d for d in os.listdir(base_folder)
                  if os.path.isdir(os.path.join(base_folder, d))]

    for folder_name in subfolders:
        finger_count = int(folder_name.split('-')[0])
        folder_path = os.path.join(base_folder, folder_name)

        image_files = glob.glob(os.path.join(folder_path, "*.jpg")) + \
                     glob.glob(os.path.join(folder_path, "*.png"))

        print(f"フォルダ {folder_name} (指の本数: {finger_count}) - 画像数: {len(image_files)}")

        for image_path in image_files:
            image_data = load_image_resized(image_path)
            if image_data is not None:
                X.append(image_data)
                y.append(finger_count)

    print(f"データセット読み込み完了: {len(X)}枚")
    return np.array(X), np.array(y)


In [ ]:
import re

def load_kaggle_dataset(kaggle_folder="/content/kaggle_fingers"):
    """Kaggleデータセットを読み込み（ファイル名からラベル抽出）"""
    print("Kaggleデータセットを読み込み中...")
    X = []
    y = []

    target_folders = ["train", "fingers"]  # 利用対象

    for folder_name in target_folders:
        folder_path = os.path.join(kaggle_folder, folder_name)
        if not os.path.exists(folder_path):
            print(f"警告: {folder_path} が存在しません")
            continue

        image_files = glob.glob(os.path.join(folder_path, "*.png")) + \
                      glob.glob(os.path.join(folder_path, "*.jpg")) + \
                      glob.glob(os.path.join(folder_path, "*.jpeg"))

        print(f"{folder_name}/: 画像数: {len(image_files)}")

        for image_path in image_files:
            filename = os.path.basename(image_path)
            match = re.search(r"_([0-5])[RL]\.", filename)
            if match:
                finger_count = int(match.group(1))
                image_data = load_image_resized(image_path)
                if image_data is not None:
                    X.append(image_data)
                    y.append(finger_count)
            else:
                print(f"ラベル抽出失敗: {filename}")

    print(f"Kaggleデータセット読み込み完了: {len(X)}枚")
    return np.array(X), np.array(y)


In [6]:
def combine_datasets(X1, y1, X2, y2):
    """2つのデータセットを結合"""
    if len(X1) == 0:
        return X2, y2
    if len(X2) == 0:
        return X1, y1

    X_combined = np.vstack([X1, X2])
    y_combined = np.hstack([y1, y2])

    print(f"結合後のデータセット: {len(X_combined)}枚")
    print(f"クラス別データ数:")
    for finger_count in sorted(set(y_combined)):
        count = np.sum(y_combined == finger_count)
        print(f"  {finger_count}本指: {count}枚")

    return X_combined, y_combined

In [ ]:
def train_combined_model():
    """結合データセットでモデルを訓練"""

    # 1. あなたのデータセットを読み込み
    # ---------------------------------
    # 実際はGoogle Driveをマウントして直接パスを通してます。
    # ---------------------------------
    your_data_folder = "/content/frames"
    X_yours, y_yours = load_your_dataset(your_data_folder)

    # 2. Kaggleデータセットを読み込み
    X_kaggle, y_kaggle = load_kaggle_dataset()

    # 3. データセットを結合
    X_combined, y_combined = combine_datasets(X_yours, y_yours, X_kaggle, y_kaggle)

    if len(X_combined) == 0:
        print("エラー: データが読み込めませんでした")
        return None

    print(f"\nモデル訓練開始")
    print(f"総データ数: {len(X_combined)}枚")

    # データ分割
    X_train, X_test, y_train, y_test = train_test_split(
        X_combined, y_combined, test_size=0.2, random_state=42, stratify=y_combined
    )

    print(f"訓練用: {len(X_train)}枚, テスト用: {len(X_test)}枚")

    # モデル訓練（より大きなデータセットに対応）
    model = RandomForestClassifier(
        n_estimators=200,  # 木の数を増加
        max_depth=15,      # 深さを増加
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )

    print("モデル訓練中...")
    model.fit(X_train, y_train)

    # 評価
    print("\nモデル評価")
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"テストセット精度: {accuracy:.3f}")
    print("\n詳細な分類レポート:")
    print(classification_report(y_test, y_pred))

    # あなたのデータだけでの性能も確認
    if len(X_yours) > 0:
        print("\nあなたのデータのみでの性能")
        your_indices = np.arange(len(X_yours))
        if len(your_indices) > 10:  # 十分なデータがある場合
            X_your_train, X_your_test, y_your_train, y_your_test = train_test_split(
                X_yours, y_yours, test_size=0.3, random_state=42
            )
            y_your_pred = model.predict(X_your_test)
            your_accuracy = accuracy_score(y_your_test, y_your_pred)
            print(f"あなたのデータでの精度: {your_accuracy:.3f}")

    return model

In [ ]:
def save_combined_model(model):
    """結合学習したモデルを保存します"""
    if model is None:
        return

    import pickle

    model_path = "/content/finger_model_combined.pkl"
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)

    print(f"\n結合学習モデルを保存しました: {model_path}")


In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

def hyperparameter_tuning(X_combined, y_combined):
    """
    過学習を防ぐためのハイパーパラメータチューニング
    """
    print("=== ハイパーパラメータチューニング開始 ===")
    
    # パラメータグリッドを定義（過学習を抑制するため）
    param_grid = {
        'n_estimators': [50, 100, 150],  # 木の数を減らして過学習を抑制
        'max_depth': [5, 10, 15],        # 深さを制限
        'min_samples_split': [10, 20, 50], # 分割に必要な最小サンプル数を増加
        'min_samples_leaf': [5, 10, 20],   # 葉ノードの最小サンプル数を増加
        'max_features': ['sqrt', 'log2']   # 特徴量の選択数を制限
    }
    
    # ベースモデル
    rf_base = RandomForestClassifier(random_state=42, n_jobs=-1)
    
    # グリッドサーチ（3-fold クロスバリデーション）
    grid_search = GridSearchCV(
        rf_base, 
        param_grid, 
        cv=3,  # データが少ない場合は3-fold
        scoring='accuracy',
        n_jobs=-1,
        verbose=1
    )
    
    print("グリッドサーチ実行中...")
    grid_search.fit(X_combined, y_combined)
    
    print(f"最適パラメータ: {grid_search.best_params_}")
    print(f"最高CVスコア: {grid_search.best_score_:.3f}")
    
    return grid_search.best_estimator_

def evaluate_model_with_cv(model, X, y):
    """
    クロスバリデーションでモデルを評価
    """
    print("\n=== クロスバリデーション評価 ===")
    
    # 5-fold クロスバリデーション
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    
    print(f"CV平均精度: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")
    print(f"各Foldの精度: {cv_scores}")
    
    return cv_scores

def plot_confusion_matrix(model, X_test, y_test):
    """
    混同行列を可視化
    """
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=range(6), yticklabels=range(6))
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    print("\n詳細な分類レポート:")
    print(classification_report(y_test, y_pred))

In [ ]:
def plot_feature_importance(model, top_n=20):
    """
    特徴量の重要度を可視化
    """
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        indices = np.argsort(importances)[::-1][:top_n]
        
        plt.figure(figsize=(10, 6))
        plt.title(f'Top {top_n} Feature Importances')
        plt.bar(range(top_n), importances[indices])
        plt.xlabel('Feature Index')
        plt.ylabel('Importance')
        plt.show()
        
        print(f"Top {top_n} 重要な特徴量のインデックス: {indices}")

def analyze_class_distribution(y):
    """
    クラス分布を分析・可視化
    """
    unique, counts = np.unique(y, return_counts=True)
    
    plt.figure(figsize=(8, 5))
    plt.bar(unique, counts)
    plt.title('Class Distribution')
    plt.xlabel('Number of Fingers')
    plt.ylabel('Count')
    for i, count in enumerate(counts):
        plt.text(unique[i], count + 10, str(count), ha='center')
    plt.show()
    
    print("クラス分布:")
    for finger_count, count in zip(unique, counts):
        percentage = count / len(y) * 100
        print(f"  {finger_count}本指: {count}枚 ({percentage:.1f}%)")
    
    # クラス不均衡のチェック
    min_count, max_count = min(counts), max(counts)
    imbalance_ratio = max_count / min_count
    print(f"\nクラス不均衡比: {imbalance_ratio:.2f}")
    if imbalance_ratio > 3:
        print("警告: クラス不均衡が大きいです。層化サンプリングやバランシング手法を検討してください。")

def save_model_with_metadata(model, accuracy, cv_scores, filename="finger_model_optimized.pkl"):
    """
    モデルとメタデータを保存
    """
    import pickle
    from datetime import datetime
    
    model_data = {
        'model': model,
        'test_accuracy': accuracy,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'cv_scores': cv_scores,
        'params': model.get_params(),
        'timestamp': datetime.now().isoformat(),
        'feature_size': model.n_features_in_ if hasattr(model, 'n_features_in_') else 'unknown'
    }
    
    with open(filename, 'wb') as f:
        pickle.dump(model_data, f)
    
    print(f"\n最適化されたモデルとメタデータを保存: {filename}")
    print(f"テスト精度: {accuracy:.3f}")
    print(f"CV平均精度: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")

In [12]:
# download_kaggle_dataset()

Kaggleデータセットをダウンロード中...
Dataset URL: https://www.kaggle.com/datasets/koryakinp/fingers
License(s): CC0-1.0
fingers.zip: Skipping, found more recently modified local copy (use --force to force download)
replace /content/kaggle_fingers/fingers/test/000e7aa6-100b-4c6b-9ff0-e7a8e53e4465_5L.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
ダウンロード完了


In [ ]:
# メイン実行
if __name__ == "__main__":
    print("=== 過学習対策付き指認識モデル訓練 ===")
    
    # データ分布の事前分析
    print("\n1. データセット読み込みと分析...")
    your_data_folder = "/content/frames"
    X_yours, y_yours = load_your_dataset(your_data_folder)
    X_kaggle, y_kaggle = load_kaggle_dataset()
    X_combined, y_combined = combine_datasets(X_yours, y_yours, X_kaggle, y_kaggle)
    
    if len(X_combined) > 0:
        print("\n2. クラス分布分析...")
        analyze_class_distribution(y_combined)
    
    # モデル訓練（ハイパーパラメータチューニング付き）
    print("\n3. モデル訓練開始...")
    optimized_model = train_combined_model()
    
    if optimized_model is not None:
        # 特徴量重要度の分析
        print("\n4. 特徴量重要度分析...")
        plot_feature_importance(optimized_model)
        
        # モデル保存（メタデータ付き）
        print("\n5. モデル保存...")
        # テスト精度とCVスコアを取得（既に計算済み）
        X_train, X_test, y_train, y_test = train_test_split(
            X_combined, y_combined, test_size=0.2, random_state=42, stratify=y_combined
        )
        test_accuracy = optimized_model.score(X_test, y_test)
        cv_scores = cross_val_score(optimized_model, X_train, y_train, cv=5)
        
        save_model_with_metadata(optimized_model, test_accuracy, cv_scores)
        
        print("\n=== 完了 ===")
        print("過学習を抑制した最適化モデルが作成されました。")
    else:
        print("エラー: モデル訓練に失敗しました。")

=== データセットを読み込み中 ===
フォルダ 0-1 (1) (指の本数: 0) - 画像数: 270
フォルダ 2-1 (指の本数: 2) - 画像数: 213
フォルダ 1-1 (指の本数: 1) - 画像数: 306
フォルダ 4-1 (指の本数: 4) - 画像数: 198
フォルダ 3-1 (指の本数: 3) - 画像数: 388
フォルダ 5-1 (指の本数: 5) - 画像数: 276
フォルダ 1-2 (指の本数: 1) - 画像数: 632
フォルダ 2-2 (指の本数: 2) - 画像数: 785
フォルダ 0-2 (指の本数: 0) - 画像数: 579
フォルダ 5-2 (指の本数: 5) - 画像数: 660
フォルダ 4-2 (指の本数: 4) - 画像数: 606
フォルダ 3-2 (指の本数: 3) - 画像数: 847
あなたのデータセット: 5760枚読み込み完了
=== Kaggleデータセットを読み込み中 ===
train/: 画像数: 18000
fingers/: 画像数: 0
Kaggleデータセット: 18000枚読み込み完了
結合後のデータセット: 23760枚
クラス別データ数:
  0本指: 3849枚
  1本指: 3938枚
  2本指: 3998枚
  3本指: 4235枚
  4本指: 3804枚
  5本指: 3936枚

=== モデル訓練開始 ===
総データ数: 23760枚
訓練用: 19008枚, テスト用: 4752枚
モデル訓練中...

=== モデル評価 ===
テストセット精度: 1.000

詳細な分類レポート:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       770
           1       1.00      1.00      1.00       787
           2       1.00      1.00      1.00       800
           3       1.00      1.00      1.00       847
           4  